## IMPORTANT: Restart the Jupyter Kernel before running the code every single time so that the current directory will reset. 

In [ ]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

import os
os.chdir(os.path.expanduser("../"))

from dees_package.youtube_functions import *

print("Current working directory:", os.getcwd())

## Functions

In [ ]:
def youtube_search(any_youtube, max_results: int, query: str, searchtype: str, region: str, category: int):
    search_data = []
    video_ids = []

    next_page_token = None

    while True :
        youtube_search_request = any_youtube.search().list(
            part="snippet",
            maxResults=min(50, max_results),  # Maximum allowed value is 50
            q=query,
            type=searchtype,
            regionCode = region,
            videoCategoryId=category,
            order="viewCount",
            fields="items(id/videoId,snippet(channelId,channelTitle,description,title)),nextPageToken,pageInfo,prevPageToken,regionCode",
            pageToken=next_page_token
        )

        # Execute the request and get the response
        youtube_search_response = youtube_search_request.execute()
        print(youtube_search_response['pageInfo']['totalResults'])

        # iterate through each element in the nested dictionary to get the relevant values of each video
        for item in youtube_search_response.get('items', []):
            video_id = item['id']['videoId']
            title = item['snippet']['title']
            channel_id = item['snippet']['channelId']
            channel_title = item['snippet']['channelTitle']

            # append the relevant values to the data dictionary to save as a dataframe
            search_data.append({
                'video_id': video_id,
                'title': title,
                'channel_id': channel_id,
                'channel_title': channel_title,
            })

            video_ids.append(video_id)

        # Check if there are more pages
        next_page_token = youtube_search_response.get('nextPageToken')
        if not next_page_token or len(video_ids) >= max_results:
            break  # No more pages or reached the desired number of results

    # Return the collected data and video IDs
    return search_data, video_ids


def get_stats(any_youtube, videoId:list):

    # video_ids_str = ','.join(videoId) # pre-2024 code, video ID as comma separated strings; but apparently it's ok to just use a list now
    video_data= []


    # create the request object
    # from the above response, we already have the channelId, channelTitle, videoID, categoryID, 
    chunk_size = 50
    for i in range(0, len(videoId), chunk_size):
        current_chunk = videoId[i:i+chunk_size-1]
        video_request = any_youtube.videos().list(
        part="statistics, id, topicDetails, contentDetails",
        id=",".join(current_chunk))

        video_response = video_request.execute()

        print(len(video_response['items']))
        pprint(video_response['items'])

    # iterate through each element in the nested dictionary to get the relevant values
        for item in video_response['items']:
            view_count = item['statistics']['viewCount'] if 'viewCount' in item['statistics'] else None
            like_count = item['statistics']['likeCount'] if 'likeCount' in item['statistics'] else None 
            comment_count = item['statistics']['commentCount'] if 'commentCount' in item['statistics'] else None
            wikipedia_category = item['topicDetails']['topicCategories'] if 'topicCategories' in item['topicDetails'] else None
            duration = item['contentDetails']['duration']


            # append the relevant values to the data dictionary to save as a dataframe
            video_data.append ({
            'video_id': item['id'],
            'view_count': view_count,
            'like_count': like_count,
            'comment_count': comment_count,
            'wikipedia_categories': wikipedia_category,
            'duration': duration
            })
            print(len(video_data))
        
    return video_data

def get_comments_in_videos(youtube, video_ids):
    """
    Get top level comments as text from all videos with given IDs (only the first 10 comments due to quote limit of Youtube API)
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with video IDs and associated top level comment in text.
    
    """
    all_comments = []
    
    for i in range (0,len(video_ids),1):
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_ids[i]
            )
            response = request.execute()
            print(response)
           
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]] 
            comments_in_video_info = {'video_id':video_ids[i], 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)

        except Exception as e:
            print(f'Could not get comments for video {i}. Error: {e}')
        continue # Skip to the next iteration in case of an error
        
    return pd.DataFrame(all_comments)  

In [ ]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [ ]:
# creating service object of the youtube version 3 API
service_youtube = build('youtube', 'v3', developerKey=credentials['youtube_api_key'])

### Getting list of music videos, carried out using .search().list() methods

In [ ]:
youtube_search_data, video_id = youtube_search(service_youtube, 2000, "official music video", "video", "US", 10)

with open('./data/yt_search_data.json', 'w') as json_file:
    json.dump(youtube_search_data, json_file, indent=4)

yt_search_df = pd.DataFrame(youtube_search_data)
yt_search_df.to_csv('./data/search.csv')


In [ ]:
test_stats = get_stats(service_youtube, video_id)

In [ ]:
len(test_stats)

### Getting statistics on each video, using video IDs from previous function as an input, carried out using .videos().list() methods

In [ ]:
video_stats = get_stats(service_youtube, video_id)  # there is a limit on the number of video ids, can only run 50 at a time. Solution: create different lists with 50 IDs each.
video_stats_df = pd.DataFrame(video_stats)

In [ ]:
video_stats_df

### Merging dataframes


In [ ]:
# merge the mv stats and search dataframes
merged_df = pd.merge(yt_search_df, video_stats_df, left_on='video_id', right_on='video_id')
merged_df.to_json('./data/merged.json')

merged_df.to_csv('./data/merged.csv')

### Getting comments, carried out using .commentThreads().list() methods

In [ ]:
comments_df = get_comments_in_videos(service_youtube, video_id) # note that comments are disabled for some videos
comments_df.head()

### Final merge of dataframes


In [ ]:
final_youtube_df = pd.merge(merged_df, comments_df, left_on='video_id', right_on='video_id', sort = False)
final_youtube_df.to_csv('./data/final_youtube.csv')
final_youtube_df